# Weekly FanDuel Game Analysis
Starting this advanced analytical process on January 12, 2022 for the Sony Open.

In [1]:
import numpy as np
import pandas as pd

Datagolf.com has a *Trending Table* that is very interesting; I'm going to attempt to optimize lineups with this and the fanduel offered golfers. Let's pull data from both locations.

Trending table can be found at https://datagolf.com/player-trends

In [2]:
# Pull Datagolf files and clean name for merging
dg = pd.read_csv('pga_data/weekly_data_files/wk2020-01-13/datagolf_trends.csv')
dg[['last','first']] = dg.player_name.str.split(',',expand=True).apply(lambda x: x.str.strip())
dg[['finish1','finish2','finish3','finish4','finish5']] = dg.last_5_finishes.str.split(' - ',expand=True)
dg = dg.drop(['last_5_finishes','player_name'], axis=1)


# Pull FanDuel data with necessary columns
fd = (pd.read_csv('pga_data/weekly_data_files/wk2020-01-13/FanDuel-PGA-2022 ET-01 ET-13 ET-70048-players-list.csv')
        .loc[:,['First Name', 'Last Name', 'FPPG', 'Salary', 'Injury Indicator']]
        .rename({'First Name':'first', 'Last Name':'last', 'Injury Indicator':'injury'}, axis=1))
fd.columns = [s.lower() for s in fd.columns]

# Merge data
df = fd.merge(dg, on=['last','first'])

# Convert to datetime
df.last_played = pd.to_datetime(df.last_played)

# Drop Players that arent playing this week
df = df[(df.playing_this_week=='pga') & (df.injury.isna())].drop(['injury','playing_this_week'],axis=1)

# Move salary to beginning of df
df = df.set_index('salary').reset_index()

# Create an average index
df['avg_index'] = df[['trend_index','baseline_index']].mean(axis=1)

# Create a rank column and differences
df[['avg_index_rank', 'salary_rank']] = df[['avg_index','salary']].rank(axis=0, ascending=False, method='first').astype('int')
df['rank_diff'] = df['salary_rank'] - df['avg_index_rank']

In [3]:
rows = 500
pd.set_option('display.max_rows', rows)
pd.set_option('display.min_rows', rows)
df[df.salary < 7200].sort_values(['avg_index'], ascending=False).reset_index(drop=True)

,salary,first,last,fppg,primary_tour,last_played,trend_index,baseline_index,finish1,finish2,finish3,finish4,finish5,avg_index,avg_index_rank,salary_rank,rank_diff
0,7000,Trey,Mullinax,57.333333,PGA,2021-11-21,0.823952,1.440551,T4,T56,CUT,CUT,CUT,1.132252,7,123,116
1,7000,Jim,Knous,83.400002,KFT,2021-10-31,-0.113927,1.310192,CUT,CUT,CUT,T11,T57,0.598132,25,111,86
2,7000,Paul,Barjon,56.325001,PGA,2021-11-21,0.017526,1.010067,T4,T42,T39,CUT,CUT,0.513796,32,104,72
3,7000,Joshua,Creel,43.466665,KFT,2021-11-21,0.297556,0.686838,T10,T8,CUT,CUT,T40,0.492197,34,107,73
4,7000,Michael,Gligic,55.700002,PGA,2021-11-21,-0.058790,0.766338,T10,T20,T66,CUT,T29,0.353774,40,109,69
5,7000,Justin,Lower,49.000000,KFT,2021-11-21,-0.016012,0.465255,T45,T15,CUT,T17,CUT,0.224622,42,115,73
6,7000,William,McGirt,59.775002,PGA,2021-11-21,-0.171457,0.501098,T45,CUT,T17,CUT,T51,0.164821,46,116,70
7,7100,Andrew,Novak,61.859998,PGA,2021-11-21,0.187500,0.084443,CUT,T58,CUT,T22,T26,0.135972,47,102,55
8,7000,Scott,Gutschewski,50.025002,PGA,2021-11-21,-0.276228,0.358316,T52,T51,CUT,T39,CUT,0.041044,53,103,50
9,7100,Jim,Herman,53.849998,PGA,2021-11-21,-0.204528,0.256114,CUT,T42,CUT,T54,T47,0.025793,55,101,46


In [61]:
mine = ['Dahmen', 'Henley', 'Im', 'Leishman']
compare = ['Conners', 'Whaley', 'Lashley', 'Spaun', 'Ancer', 'McNealy', 'Herman', 'Power']

df[df['last'].isin(mine+compare)]

,salary,first,last,fppg,primary_tour,last_played,trend_index,baseline_index,finish1,finish2,finish3,finish4,finish5,avg_index,avg_index_rank,salary_rank,rank_diff
3,11500,Sungjae,Im,118.740002,PGA,2022-01-09,2.124830,0.882860,T31,1,T9,T19,T8,1.503845,1,4,3
4,11300,Abraham,Ancer,89.420001,PGA,2022-01-09,0.671660,-0.567273,T14,T7,T27,T14,T35,0.052193,52,5,-47
5,11200,Marc,Leishman,110.759998,PGA,2022-01-09,1.908727,0.928154,T4,T3,T38,T19,T10,1.418440,3,6,3
7,11000,Corey,Conners,94.866669,PGA,2021-11-21,1.183273,-0.190299,T22,T22,T17,T40,T22,0.496487,33,8,-25
13,10400,Seamus,Power,83.642857,PGA,2022-01-09,1.355633,0.632898,T12,T11,CUT,T4,T15,0.994266,10,14,4
14,10300,Russell,Henley,91.620001,PGA,2021-11-21,1.315091,0.321504,T21,T25,T56,T7,T22,0.818297,15,15,0
21,9800,Maverick,McNealy,82.133331,PGA,2021-11-14,1.051365,0.366201,CUT,T38,T25,T11,T19,0.708783,20,22,2
27,9500,Joel,Dahmen,92.133331,PGA,2022-01-09,0.986483,0.686899,T24,T45,T5,T29,T28,0.836691,14,28,14
67,8000,J.J.,Spaun,73.057142,PGA,2021-11-21,1.073811,1.594877,T35,T7,T27,T46,T16,1.334344,4,68,64
81,7600,Nate,Lashley,68.500000,PGA,2021-11-21,0.433231,0.728240,T17,T35,CUT,CUT,T51,0.580736,26,82,56


### Notes
- Conners is predicted to do well by many experts, but not in this model. Check at the end of the week who is correct
